<a href="https://colab.research.google.com/github/asmi281999/Clssification_model_for_claim_description/blob/main/Classification_model_for_claim_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

asmitabose_dataset_public_path = kagglehub.dataset_download('asmitabose/dataset-public')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-public/Dataset_Public.xlsx


### Library

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import re
import string
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import pickle
import numpy as np
import time

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

In [ ]:
import re
from bs4 import BeautifulSoup

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Dataset

In [ ]:
df=pd.read_excel('/kaggle/input/dataset-public/Dataset_Public.xlsx')
df.head(3)

,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
1,CLAIMANT ALLEGES SHE SUFFERED INJURIES IN AN E...,GB,Elevator/Escalator
2,"IV PASSENGER SUSTAINED INJURIES, OV AND IV COL...",AB,Sideswipe or lane change


### Preprocessing

In [ ]:
df.columns

Index(['Claim Description', 'Coverage Code', 'Accident Source'], dtype='object')

#### 1. Duplicate removal

In [ ]:
df['Claim Description'].duplicated().sum()

27699

In [ ]:
df['Claim Description'].value_counts()

Claim Description
CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DAMAGE.                                                                                                                                                                            1057
ROCK FROM ROAD - NO ONE AT FAULT                                                                                                                                                                                                598
CLAIMANT DROVE OVER A POTHOLE CAUSING VEHICLE DAMAGE.                                                                                                                                                                           432
PLAINTIFF ALLEGES INJURY CAUSED BY EMBEDDED BROKEN ARM OF PARAGARDIUD.                                                                                                                                                          311
WATER DAMAGE                                                          

In [ ]:
df[df['Claim Description']=='CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DAMAGE.']

,Claim Description,Coverage Code,Accident Source
296,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
319,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
589,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
667,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
763,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
...,...,...,...
191123,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
191265,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
191523,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole
191550,CLAIMANT DROVE OVER POTHOLE CAUSING VEHICLE DA...,GD,Pothole


In [ ]:
df.drop_duplicates(subset = ['Claim Description'],inplace = True)

In [ ]:
df

,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
1,CLAIMANT ALLEGES SHE SUFFERED INJURIES IN AN E...,GB,Elevator/Escalator
2,"IV PASSENGER SUSTAINED INJURIES, OV AND IV COL...",AB,Sideswipe or lane change
3,CLAIMANT ALLEGES SHE WAS BURNED UNKNOWN DEGREE...,PA,Food Product
4,THE IV WAS MERGING INTO A CONSTRUCTION ZONE WH...,AD,Struck vehicle in rear
...,...,...,...
191683,PACKAGE CAR LEAKED OIL ON CONCRETE DRIVEWAY,AD,Oil or grease
191685,SOME WOOD FELL OVER AND HIT GUESTS LEGS,GB,Alleged lack of security
191686,FEMALE FELL DOWN ON PLATFORM. ...,AB,Not Otherwise Classified
191687,IWAS DRIVING DOWNHILL IN CUSTOMER YARD ON WET ...,AD,Sideswipe or lane change


#### 2. Text preprocessing

In [ ]:
def preprocess(qs):

  # 1.Replace all upper case letter to lower case

  qs=str(qs).lower().strip()

  # 2. Replace certain special characters with their string equivalents

  qs= qs.replace('!' , 'exc')
  qs= qs.replace('@' , "at ")
  qs= qs.replace('#' , 'hash')
  qs= qs.replace('$' , 'dollar')
  qs= qs.replace('%' ,'percent')
  qs= qs.replace('₹', ' rupee ')
  qs= qs.replace('€', ' euro ')
  qs= qs.replace('^' , 'upar')
  qs= qs.replace('&' , 'amp')
  qs= qs.replace('*' , 'star')
  qs= qs.replace('-' , 'minus')
  qs= qs.replace('+' , 'plus')
  qs= qs.replace('=' , 'equal')
  qs= qs.replace('_' , 'underscore')
  qs= qs.replace('/' , 'slash')
  qs= qs.replace('?' , 'question')
  qs= qs.replace('<' , 'lessthan')
  qs= qs.replace('>' , 'greaterthan')

  # 3. Replace pattern '[Math]' as it has appeared 900 times in the whole dataset.

  qs= qs.replace('[math]' , 'math')

  # 4. Replacing some numbers with their equivalent character (not accurate)

  qs = qs.replace(',000,000,000 ', 'b ')
  qs = qs.replace(',000,000 ', 'm ')
  qs = qs.replace(',000 ', 'k ')
  qs = re.sub(r'([0-9]+)000000000', r'\1b', qs)
  qs = re.sub(r'([0-9]+)000000', r'\1m', qs)
  qs = re.sub(r'([0-9]+)000', r'\1k', qs)


  # 5. Decontracting words
  # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
  # https://stackoverflow.com/a/19794953
  contractions ={
      "a'ight" : "alright",
      "ain't" :	 "am not or is not or are not or has not or have not or did not",
      "amn't" :	"am not",
      "n" : "and",
      "arencha" : "are not you",
      "aren’t you" : "are not you",
      "aren't": "are not",
      "‘bout" :	"about",
      "can't" :	"cannot",
      "cap’n" :	"captain",
      "'cause" : "because",
      "’cept" :	"except",
      "could've" :	"could have",
      "couldn't" :	"could not",
      "couldn't've" :	"could not have",
      "cuppa" :	"cup of",
      "dammit" : "damn it",
      "daren't" :	"dare not or dared not",
      "daresn't" : "dare not",
      "dasn't" :	"dare not",
      "didn't" :	"did not",
      "doesn't" :	"does not",
      "don't" :	"do not or does not",
      "dunno" :	"do not know",
      "don't know" : "do not know",
      "d'ye" : 	"do you or did you",
      "d'ya" :	"do you or did you",
      "e'en" : 	"even",
      "e'er" : 	"ever",
      "'em" :	"them",
      "everybody's" :	"everybody is",
      "everyone's" :	"everyone is",
      "finna" :	"fixing to",
      "fo’c’sle" : "forecastle",
      "’gainst" :	"against",
      "g'day" : "good day",
      "gimme" : "give me",
      "giv'n" : "given",
      "gi'z" :	"give us or give me",
      "gonna" :	"going to",
      "gon't" : "go not",
      "gotta" :	"got to",
      "hadn't" :	"had not",
      "had've" :	"had have",
      "hasn't" :	"has not",
      "haven't" :	"have not",
      "he'd" :	"he had or he would",
      "he'll" :	"he shall or he will",
      "helluva" : "hell of a",
      "he's" :	"he has or he is",
      "here's" :	"here is",
      "how'd" :	"how did or how would",
      "howdy" : "how do you do or how do you fare",
      "how'll" : "how will",
      "how're" :	"how are",
      "how's" :	"how has or how is or how does",
      "I'd" :	"I had or I would",
      "I'd've" :	"I would have",
      "I'd'nt" :	"I would not",
      "I'd'nt've" : "I would not have",
      "I'll" :	"I shall or I will",
      "I'm" :	"I am",
      "Imma" : "I am about to or I am going to",
      "I'm'o" :	"I am going to",
      "innit" : "isn't it",
      "Ion" : "I don't or I do not",
      "I've" :	"I have",
      "isn't" :	"is not",
      "it'd" :	"it would",
      "it'll" :	"it shall or it will",
      "it's" :	"it has or it is",
      "Idunno" : "I donot know",
      "kinda" : "kind of",
      "let's" :	"let us",
      "loven't" :	"love not",
      "ma'am " :	"madam",
      "mayn't" :	"may not",
      "may've" :	"may have",
      "methinks" :	"I think",
      "mightn't" :	"might not",
      "might've" :	"might have",
      "mustn't" :	"must not",
      "mustn't've" :	"must not have",
      "must've"	: "must have",
      "‘neath" :	"beneath",
      "needn't" :	"need not",
      "nal" :	"and all",
      "ne'er" : "never",
      "o'clock" :	"of the clock",
      "o'er" :	"over",
      "ol'" :	"old",
      "ought've" :	"ought have",
      "oughtn't" :	"ought not",
      "oughtn't've" :	"ought not have",
      "‘round" :	"around",
      "'s" :	"is or has or does or us",
      "shalln't" :	"shall not",
      "shan't" : "shall not",
      "she'd" :	"she had or she would",
      "she'll" :	"she shall or she will",
      "she's" :	"she has or she is",
      "should've" :	"should have",
      "shouldn't" :	"should not",
      "shouldn't've" :	"should not have",
      "somebody's" :	"somebody has or somebody is",
      "someone's" :	"someone has or someone is",
      "something's"	 : "something has or something is",
      "so're" :	"so are",
      "so’s" : "so is or so has",
      "so’ve" :	"so have",
      "that'll" : "that shall or that will",
      "that're" : "that are",
      "that's" :	"that has or that is",
      "that'd" :	"that would or that had",
      "there'd" :	"there had or there would",
      "there'll" :	"there shall or there will",
      "there're" :	"there are",
      "there's" :	"there has or there is",
      "these're" :	"these are",
      "these've" :	"these have",
      "they'd" :	"they had or they would",
      "they'll" :	"they shall or they will",
      "they're" :	"they are or they were",
      "they've" :	"they have",
      "this's"	: "this has or this is",
      "those're" :	"those are",
      "those've" : 	"those have",
      "'thout" : 	"without",
      "’til" :	"until",
      "'tis" : "it is",
      "to've" : "to have",
      "'twas" : "it was",
      "'tween" : 	"between",
      "'twere" :	"it were",
      "w'all"	: "we all",
      "w'at" :	"we at",
      "wanna" :	"want to",
      "wasn't" :	"was not",
      "we'd" :	"we had or we would or we did",
      "we'd've" :	"we would have",
      "we'll" :	"we shall or we will",
      "we're" :	"we are",
      "we've" :	"we have",
      "weren't" :	"were not",
      "whatcha" :	"what are you or what about you ",
      "what'd" :	"what did",
      "what'll" :	"what shall or what will",
      "what're"	: "what are or what were",
      "what's" :	"what has or what is or what does",
      "what've" :	"what have",
      "when's" :	"when has or when is",
      "where'd" :	"where did",
      "where'll" :	"where shall or where will",
      "where're" :	"where are",
      "where's" :	"where has or where is or where does",
      "where've" :	"where have",
      "which'd" :	"which had or which would",
      "which'll" :	"which shall or which will",
      "which're" :	"which are",
      "which's" :	"which has or which is",
      "which've" :	"which have",
      "who'd" :	"who would or who had or who did",
      "who'd've" :	"who would have",
      "who'll" :	"who shall or who will",
      "who're" :	"who are",
      "who's" :	"who has or who is or who does",
      "who've" :	"who have",
       "why'd" :	"why did",
      "why're" :	"why are",
      "why's" :	"why has or why is or why does",
      "willn't" :	"will not",
      "won't" :	"will not",
      "wonnot" :	"will not",
      "would've" :	"would have",
      "wouldn't" :	"would not",
      "wouldn't've" :	"would not have",
      "y'ain't" :	"you are not or you have not or you did not",
      "y'all" :	"you all",
      "y'all'd've" : 	"you all would have",
      "y'all'd'n't've" :	"you all would not have",
      "y'all're" :	"you all are",
      "y'all'ren't" :	"you all are not",
      "y'at" :	"you at",
      "yes’m" :	"yes ma’am or yes madam",
      "y'know" :	"you know",
      "yessir":	"yes sir",
      "you'd" :	"you had or you would",
      "you'll" :	"you shall or you will",
      "you're" : "you are",
      "you've" : "you have",
      "when'd" :	"when did",
      "willn't" :	"will not",
  }

  qs_decontracted = []

  for word in qs.split():
      if word in contractions:
          word = contractions[word]

      qs_decontracted.append(word)

  qs = ' '.join(qs_decontracted)
  qs = qs.replace("'ve", " have")
  qs = qs.replace("n't", " not")
  qs = qs.replace("'re", " are")
  qs = qs.replace("'ll", " will")

  # 6. Removing HTML tags
  qs = BeautifulSoup(qs)
  qs = qs.get_text()

  # 7. Remove punctuations
  pattern = re.compile('\W')
  qs = re.sub(pattern, ' ', qs).strip()


  return qs



In [ ]:
df['Claim Description'].apply(preprocess)

0         the iv was making a left turn on a green arrow...
1         claimant alleges she suffered injuries in an e...
2         iv passenger sustained injuries  ov and iv col...
3         claimant alleges she was burned unknown degree...
4         the iv was merging into a construction zone wh...
                                ...                        
191683          package car leaked oil on concrete driveway
191685              some wood fell over and hit guests legs
191686      female fell down on platform  bus reg   bk63ywl
191687    iwas driving downhill in customer yard on wet ...
191688    donna george was walking up to register 2 and ...
Name: Claim Description, Length: 163991, dtype: object

#### 3. Null value removal

In [ ]:
df.isnull().sum().sum()

1

In [ ]:
df = df.dropna(axis=0)


In [ ]:
df.columns

Index(['Claim Description', 'Coverage Code', 'Accident Source'], dtype='object')

### Target Variables

#### 1. Coverage Code

In [ ]:
df['Coverage Code'].nunique()

43

In [ ]:
df['Coverage Code'].value_counts()

Coverage Code
AD    43070
GB    32193
GD    22399
AP    18894
AL     9983
AB     9397
PA     8030
PB     5089
RB     4553
NS     2297
PM     1359
AU     1273
EL      855
GK      632
AN      558
RC      514
PI      384
GO      337
PL      317
DC      261
GL      237
AM      224
EP      184
IK      183
OM      138
EO      137
PP      112
TE       85
BM       66
LL       57
IM       40
BL       36
CM       30
FB       18
OI       14
RF       12
BR        6
EB        4
RQ        3
FF        3
LS        3
EI        2
PC        1
Name: count, dtype: int64

In [ ]:
c_c_classes = pd.DataFrame(df['Coverage Code'].value_counts())

In [ ]:
c_c_classes

,count
Coverage Code,
AD,43070
GB,32193
GD,22399
AP,18894
AL,9983
AB,9397
PA,8030
PB,5089
RB,4553


#### 1. Data Imbalance

In [ ]:
few_classes = c_c_classes[c_c_classes['count'] < 10].index.to_list()
medium_classes = c_c_classes[c_c_classes['count'] < 1000].index.to_list()

In [ ]:
few_classes

['BR', 'EB', 'RQ', 'FF', 'LS', 'EI', 'PC']

In [ ]:
fewer_df = df[df['Coverage Code'].isin(few_classes)]

fewer_df.head(2)

,Claim Description,Coverage Code,Accident Source
33833,THE CALLER STATES THAT THE BUILDER CALLED AND ...,BR,"Burglary, Robbery, Theft - Property"
63005,OIL DISCOVERED IN MONITORING TANKS,EI,Fuel


In [ ]:
fewer_df.shape

(22, 3)

In [ ]:
# for fewer classes either we can directrly create dictionary and use classes


fewer_class_dictionary = dict()
for idx,row in fewer_df.iterrows():

    fewer_class_dictionary[row[0]] = [row[1],row[2]]

fewer_class_dictionary

{'THE CALLER STATES THAT THE BUILDER CALLED AND STATED THE ITEMS    FROM THE PROPERTY THAT WERE BEING USED TO BUILD THE HOME WERE     STOLEN AND POLICE REPORT WAS CREATED.': ['BR',
  'Burglary, Robbery, Theft - Property'],
 'OIL DISCOVERED IN MONITORING TANKS': ['EI', 'Fuel'],
 "EEOC CHARGE OF DISCRIMINATION FILED BY CURRENT ST. JOHN'S         UNIVERSITY EMPLOYEE CHARLES FORTMANN": ['EB',
  'Alleged Discrimination'],
 'ALLEGATIONS OF VIOLATION OF WAGE AND HOUR VIOLATIONS; LIMITED     ASSIGNMENT FOR PAYMENT OF LEGAL FEES.': ['EB',
  'Alleged Negligent Act'],
 'GRAFFITI PAINTED ON WALL.': ['RQ', 'Other - comprehensive'],
 'SEWAGE RELEASE': ['EI', 'Backup Sewer, Seepage'],
 'CYBER BREACH': ['FF', 'Not Otherwise Classified'],
 'CLAIMANT SUFFERED DEFAMATION AND LOSS OF BUSINESS': ['LS',
  'Loss of Use, Business Interruption'],
 'PROPERTY THEFT AND DAMAGE': ['BR', 'Burglary, Robbery, Theft - Property'],
 'TOOLS WERE STOLEN FROM A JOB SITE AT FRANCIS HOWELL NORTH': ['BR',
  'Burglary, Robbery

In [ ]:
medium_df=df[df['Coverage Code'].isin(medium_classes)]
medium_df.head(2)

,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
9,CLAIMANT SUFFERED BODILY INJURY DUE TO PICKED ...,EL,Fall - other falls


In [ ]:
# Filter out rows that are in fewer_df
final_df = df[~df['Coverage Code'].isin(few_classes)]


In [ ]:
final_df.shape

(163968, 3)

In [ ]:
df_2 = final_df.copy()

def convert_class(feature):
    if feature in medium_classes:
        return 'med'
    else:
        return feature

final_df['Coverage Code'] = final_df['Coverage Code'].apply(convert_class)

In [ ]:
final_df['Coverage Code'].value_counts()

Coverage Code
AD     43070
GB     32193
GD     22399
AP     18894
AL      9983
AB      9397
PA      8030
med     5431
PB      5089
RB      4553
NS      2297
PM      1359
AU      1273
Name: count, dtype: int64

#### 2. Model - stage 1

In [ ]:
# create features and class label

X = final_df['Claim Description']

y = final_df['Coverage Code']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
count_vector_step_1 = CountVectorizer()

X_train = count_vector_step_1.fit_transform(X_train)
X_test = count_vector_step_1.transform(X_test)

In [ ]:
model_1 = LogisticRegression(random_state=42)
model_1.fit(X_train, y_train)
y_pred = model_1.predict(X_test)


In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))


Confusion Matrix:
[[ 584  937    6  167    9  114   19    5    2    0    0    2   36]
 [ 305 6916  120  844   57   60  239    3    1   12    0   42   62]
 [   1  131 1884   22    0    1   11    0    0    1    0    1    7]
 [  64 1541   15 1818    6   27  155    7    0    1    0   28   27]
 [  18  109    0   24   85    4    4    3    0    0    0    0    2]
 [  42   60    0    9    2 5789  219   28  122   19   23   18   65]
 [   7  280   13   76    2  315 3490    4   12  140    1   97   63]
 [   4   16    0   12    0   96    4  352    5    0    0    0   10]
 [   1    5    0    0    0  189   17    5 1332   40    0    0    3]
 [   1    9    0    3    0   29  159    0   41  754    0   20   11]
 [   1    4    0    0    0  207    8    0   13    0   32    0    0]
 [   0   84    4   35    0   31  170    0    0   25    1  536   13]
 [  45  166   13   86    1  230  153   12   10    8    0   50  303]]

Classification Report:
              precision    recall  f1-score   support

          AB      

In [ ]:
target_names = model_1.classes_
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

          AB       0.54      0.31      0.40      1881
          AD       0.67      0.80      0.73      8661
          AL       0.92      0.92      0.92      2059
          AP       0.59      0.49      0.54      3689
          AU       0.52      0.34      0.41       249
          GB       0.82      0.91      0.86      6396
          GD       0.75      0.78      0.76      4500
          NS       0.84      0.71      0.77       499
          PA       0.87      0.84      0.85      1592
          PB       0.75      0.73      0.74      1027
          PM       0.56      0.12      0.20       265
          RB       0.68      0.60      0.63       899
         med       0.50      0.28      0.36      1077

    accuracy                           0.73     32794
   macro avg       0.69      0.60      0.63     32794
weighted avg       0.72      0.73      0.72     32794



In [ ]:
# save model
path = "/kaggle/working/logistic_regression_cc_1st_stg.pickle"
pickle.dump(model_1, open(path, "wb"))
print("Model saved to", path)


Model saved to /kaggle/working/logistic_regression_cc_1st_stg.pickle


#### Model - stage 2

In [ ]:
med_df = df_2[df_2['Coverage Code'].isin(medium_classes[:-5])]

med_df.head(2)

,Claim Description,Coverage Code,Accident Source
0,THE IV WAS MAKING A LEFT TURN ON A GREEN ARROW...,AN,"Struck pedestrian, bicycle"
9,CLAIMANT SUFFERED BODILY INJURY DUE TO PICKED ...,EL,Fall - other falls


In [ ]:
med_df.shape

(5431, 3)

In [ ]:
med_df['Coverage Code'].value_counts()

Coverage Code
EL    855
GK    632
AN    558
RC    514
PI    384
GO    337
PL    317
DC    261
GL    237
AM    224
EP    184
IK    183
OM    138
EO    137
PP    112
TE     85
BM     66
LL     57
IM     40
BL     36
CM     30
FB     18
OI     14
RF     12
Name: count, dtype: int64

In [ ]:
# create features and class label

X = med_df['Claim Description']

y = med_df['Coverage Code']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify = y)

In [ ]:
count_vector_step_2 = CountVectorizer()

X_train = count_vector_step_2.fit_transform(X_train)
X_test = count_vector_step_2.transform(X_test)

In [ ]:
model_2 = LogisticRegression(random_state=42)
model_2.fit(X_train, y_train)
y_pred = model_2.predict(X_test)


In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))

Confusion Matrix:
[[ 22  14   0   0   0   1   4   0   0   0   0   0   0   0   0   1   0   0
    0   1   0   2   0   0]
 [ 16  84   0   0   0   2   3   0   0   0   4   0   0   0   0   0   0   0
    0   2   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0   0   2   0   1   0   0   0   0   0
    0   0   0   2   0   0]
 [  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0
    0   1   0  11   0   0]
 [  0   0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   4   0   0]
 [  3   5   0   0   0  38   0   0   0   0   4   1   0   0   0   0   0   0
    0   1   0   0   0   0]
 [  0   5   0   0   0   0 159   0   0   0   0   0   1   0   0   0   0   0
    1   1   0   4   0   0]
 [  0   0   0   0   0   0   0  13   7   0   0   0   0   0   0   0   0   1
    1   5   0   0   0   0]
 [  0   0   0   0   0   0   1   2  27   0   0   0   1   0   0   0   0   0
    5   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

In [ ]:
target_names = model_2.classes_
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

          AM       0.51      0.49      0.50        45
          AN       0.69      0.75      0.72       112
          BL       0.00      0.00      0.00         7
          BM       0.00      0.00      0.00        13
          CM       0.00      0.00      0.00         6
          DC       0.90      0.73      0.81        52
          EL       0.86      0.93      0.89       171
          EO       0.57      0.48      0.52        27
          EP       0.66      0.73      0.69        37
          FB       0.00      0.00      0.00         4
          GK       0.75      0.79      0.77       127
          GL       0.90      0.74      0.81        47
          GO       0.74      0.62      0.67        68
          IK       0.92      0.97      0.95        37
          IM       0.00      0.00      0.00         8
          LL       0.60      0.27      0.37        11
          OI       1.00      0.33      0.50         3
          OM       0.75    

In [ ]:
# save model
path = "/kaggle/working/logistic_regression_cc_2nd_stg.pickle"
pickle.dump(model_2, open(path, "wb"))
print("Model saved to", path)


Model saved to /kaggle/working/logistic_regression_cc_2nd_stg.pickle


#### Prediction

In [ ]:
# load the different models from path

step_1_model_path = "/kaggle/working/logistic_regression_cc_1st_stg.pickle"
step_2_model_path = "/kaggle/working/logistic_regression_cc_2nd_stg.pickle"

step_1_model = pickle.load(open(step_1_model_path, 'rb'))
step_2_model = pickle.load(open(step_2_model_path, 'rb'))


In [ ]:
# create predict function for predicting class label

def predict(model_1,model_2,final_dict,query):
    # predict

    test_1 =  count_vector_step_1.transform([query])
    y_pred = model_1.predict(test_1)
    if y_pred == 'med':
        test_2 =  count_vector_step_2.transform([query])
        y_pred = model_2.predict(test_2)
    else:
        y_pred = y_pred

    if query in final_dict.keys():
        y_pred = final_dict[query]
    else:
        y_pred = y_pred

    return y_pred[0]

In [ ]:
# check predictions

final_result= []

test = df.sample(150)
test['Claim Description'].apply(preprocess)
x_test = test['Claim Description']
for query in x_test:

    result = predict(step_1_model,step_2_model,fewer_class_dictionary,query)
    final_result.append(result)

In [ ]:
# check the accuracy and other metrics

y_test_ = test['Coverage Code'].tolist()

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for logistic regression :',np.round(accuracy,2))

print("*"*100)


print(classification_report(y_test_, final_result))
print('*'*100)

Accuracy for logistic regression : 0.72
****************************************************************************************************
              precision    recall  f1-score   support

          AB       0.80      0.57      0.67         7
          AD       0.62      0.78      0.69        32
          AL       1.00      0.90      0.95        10
          AP       0.76      0.66      0.70        29
          AU       1.00      1.00      1.00         1
          GB       0.73      0.95      0.83        20
          GD       0.64      0.70      0.67        20
          GK       0.00      0.00      0.00         0
          IK       0.00      0.00      0.00         1
          LL       0.00      0.00      0.00         1
          NS       1.00      0.67      0.80         3
          PA       0.80      0.80      0.80        10
          PB       1.00      0.80      0.89         5
          PI       0.00      0.00      0.00         1
          PL       0.00      0.00      0.00     

#### 2. Accident Source

In [ ]:
df['Accident Source'].nunique()

312

In [ ]:
final_df['Accident Source'].value_counts()

Accident Source
Alleged Negligent Act            20251
Sideswipe or lane change         15414
Struck animal or object           7720
Struck vehicle in rear            7400
Not Otherwise Classified          7376
                                 ...  
Multiple Vehicle Collision          14
Machine - point of operation        14
VEHICLE                             13
Defective Pipework                  11
Boiler, pressure vessel, etc.       10
Name: count, Length: 312, dtype: int64

In [ ]:
# check count of classes

a_s_classes = pd.DataFrame(df_2['Accident Source'].value_counts())

In [ ]:
a_s_classes

,count
Accident Source,
Alleged Negligent Act,20251
Sideswipe or lane change,15414
Struck animal or object,7720
Struck vehicle in rear,7400
Not Otherwise Classified,7376
...,...
Multiple Vehicle Collision,14
Machine - point of operation,14
VEHICLE,13


#### Data imbalance

In [ ]:
few_classes_as = a_s_classes[a_s_classes['count'] < 20].index.to_list()
medium_classes_as = a_s_classes[a_s_classes['count'] < 100].index.to_list()

In [ ]:
# check the data with very few classes

fewer_df_as = df_2[df_2['Accident Source'].isin(few_classes_as)]

fewer_df_as.head(2)

,Claim Description,Coverage Code,Accident Source
485,CLAIMANT SUFFER PROPERTY DAMAGE DUE TO LEAKAGE.,GD,Leak/Flood - Defective Materials
545,A CUSTOMER WAS BUYING BLEACH AT ACO AROUND 4:0...,GD,Container - Packaging Etc


In [ ]:
# for fewer classes either we can directrly create dictionary and use classes


fewer_class_dictionary_as = dict()
for idx,row in fewer_df_as.iterrows():

    fewer_class_dictionary_as[row[0]] = [row[1],row[2]]

fewer_class_dictionary_as

{'CLAIMANT SUFFER PROPERTY DAMAGE DUE TO LEAKAGE.': ['GD',
  'Leak/Flood - Defective Materials'],
 "A CUSTOMER WAS BUYING BLEACH AT ACO AROUND 4:00 PM, AND THE LID   WASN'T SEALED PROPERLY AND LEAKED ON HIS PANT WHILE HE WAS        SCANNING PRODUCTS.": ['GD',
  'Container - Packaging Etc'],
 'CLAIMANT WAS DRIVING ON THE ROAD AND VEHICLE HIT THE SPEED BUMP   CAUSING VEHICLE DAMAGE.': ['GD',
  'Speed bump'],
 'CLAIMANT SUFFERED PROPERTY DAMAGE DUE TO FLOOD.': ['GD',
  'Leak/Flood - Defective Workmanship'],
 "DEVIL'S POOL HOT TUB - TRACEY GOT A RASH ON TORSO AND BACK ALONG  WITH CHILLS AND BODY ACHES, ANTIBIOTICS WERE REQUIRED": ['GB',
  'Infectious, parasitic agents, NOC - Liab.'],
 'CLAIMANT SUFFERED PROPERTY DAMAGED DUE TO DAMPNESS.': ['GD',
  'Leak/Flood - Defective Workmanship'],
 'ACCIDENT ENTRE 2 VOITURE DERRI RE LE MAGASIN TROP GLISSANT, ALORS NE POUVAIS PAS FREINER.': ['AD',
  'Frost/Ice/Snow'],
 'CLAIMANT FALL DUE TO KERB AND SUFFER THE INJURIES': ['GB',
  'Trip - kerb stone'],


In [ ]:
df_as_2 = df_2.copy()

def convert_class(feature):
    if feature in medium_classes_as:
        return 'med'
    else:
        return feature

df_as_2['Accident Source'] = df_as_2['Accident Source'].apply(convert_class)

In [ ]:
df_as_2['Accident Source'].value_counts()

Accident Source
Alleged Negligent Act                    20251
Sideswipe or lane change                 15414
Struck animal or object                   7720
Struck vehicle in rear                    7400
Not Otherwise Classified                  7376
                                         ...  
Insured vehicle causes other accident      103
Excavation, trench, tunnel                 102
Malicious Mischief                         101
Door - Office                              101
Alleged Discrimination                     100
Name: count, Length: 150, dtype: int64

#### Model

In [ ]:
# create features and class label

X = df_as_2['Claim Description']

y = df_as_2['Accident Source']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# convert data into vector form

count_vector_step_1 = CountVectorizer()

X_train = count_vector_step_1.fit_transform(X_train)
X_test = count_vector_step_1.transform(X_test)

In [ ]:
model_3 = LogisticRegression(random_state=42)
model_3.fit(X_train, y_train)
y_pred = model_3.predict(X_test)


In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))

Confusion Matrix:
[[ 12   0   0 ...   0   0   1]
 [  0   6   1 ...   0   0   0]
 [  0   3   4 ...   0   0   1]
 ...
 [  0   0   0 ...   0   0   3]
 [  0   0   0 ...   0 384   4]
 [  1   1   1 ...   0   8 406]]

Classification Report:
                                            precision    recall  f1-score   support

    Accidental disposal of property/object       0.19      0.12      0.15        97
                   Administrative Decision       0.46      0.29      0.35        21
                    Alleged Discrimination       0.67      0.19      0.30        21
                          Alleged Incident       0.00      0.00      0.00        22
      Alleged Misconduct - Personal Injury       0.58      0.29      0.39        24
                     Alleged Negligent Act       0.37      0.60      0.46      4058
                   Alleged concrete defect       0.00      0.00      0.00        24
         Alleged contamination or spoilage       0.51      0.51      0.51       389
      All

In [ ]:
target_names = model_3.classes_
print(classification_report(y_test, y_pred, target_names=target_names))

                                            precision    recall  f1-score   support

    Accidental disposal of property/object       0.19      0.12      0.15        97
                   Administrative Decision       0.46      0.29      0.35        21
                    Alleged Discrimination       0.67      0.19      0.30        21
                          Alleged Incident       0.00      0.00      0.00        22
      Alleged Misconduct - Personal Injury       0.58      0.29      0.39        24
                     Alleged Negligent Act       0.37      0.60      0.46      4058
                   Alleged concrete defect       0.00      0.00      0.00        24
         Alleged contamination or spoilage       0.51      0.51      0.51       389
      Alleged damage to property of others       0.44      0.20      0.28       372
               Alleged design flaw, defect       0.48      0.49      0.49       345
         Alleged foreign object in product       0.62      0.74      0.67  

In [ ]:
# save model
path = "/kaggle/working/logistic_regression_as_1st_stg.pickle"
pickle.dump(model_3, open(path, "wb"))
print("Model saved to", path)


Model saved to /kaggle/working/logistic_regression_as_1st_stg.pickle


#### Prediction

In [ ]:
step_1_model_path = "/kaggle/working/logistic_regression_as_1st_stg.pickle"
# step_2_model_path = "/kaggle/working/logistic_regression_as_2nd_stg.pickle"

step_1_model = pickle.load(open(step_1_model_path, 'rb'))
# step_2_model = pickle.load(open(step_2_model_path, 'rb'))



In [ ]:
# create predict function for predicting class label

def predict(model_1,final_dict,query):
    # predict

    test_1 =  count_vector_step_1.transform([query])
    y_pred = model_1.predict(test_1)
    # if y_pred == 'med':
    #     test_2 =  count_vector_step_2.transform([query])
    #     y_pred = model_2.predict(test_2)
    # else:
    y_pred = y_pred

    if query in final_dict.keys():
        y_pred = final_dict[query]
    else:
        y_pred = y_pred

    return y_pred[0]



In [ ]:

  # check predictions

final_result= []

test = df.sample(150)
test['Claim Description'].apply(preprocess)
x_test = test['Claim Description']
for query in x_test:

    result = predict(step_1_model,fewer_class_dictionary,query)
    final_result.append(result)


In [ ]:
# check the accuracy and other metrics

y_test_ = test['Accident Source'].tolist()

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy for logistic regression :',np.round(accuracy,2))

print("*"*100)


print(classification_report(y_test_, final_result))
print('*'*100)

Accuracy for logistic regression : 0.46
****************************************************************************************************
                                            precision    recall  f1-score   support

                     Alleged Negligent Act       0.38      0.93      0.54        14
         Alleged contamination or spoilage       1.00      1.00      1.00         6
      Alleged damage to property of others       0.00      0.00      0.00         2
         Alleged foreign object in product       0.50      1.00      0.67         1
              Alleged improper formulation       0.00      0.00      0.00         1
      Alleged improper maintenance - other       1.00      0.20      0.33         5
                 Alleged product loss, NOC       1.00      1.00      1.00         1
Alleged service or maint.- vendor/retailer       0.00      0.00      0.00         0
Alleged service or maintenance by customer       0.00      0.00      0.00         1
                  

In [ ]:
!pip install streamlit

import streamlit as st
import pandas as pd
import joblib
import re
import string
from string import digits
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from io import BytesIO
import base64

# Streamlit App Title
st.title("Gallagher: Logistic Regression-based Text Classification and Excel Processing App")

# File upload for Excel file
uploaded_file = st.file_uploader("Upload an Excel file", type=["xlsx"])

# Function for file download
def get_binary_file_downloader_link(file_data, file_name, link_text):
    excel_buffer = BytesIO()
    file_data.to_excel(excel_buffer, index=False, engine='xlsxwriter')
    b64 = base64.b64encode(excel_buffer.getvalue()).decode()
    href = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="{file_name}">{link_text}</a>'
    return href



# Load Pretrained Models and Vectorizers
logistic_model_1 = joblib.load("/kaggle/working/logistic_regression_cc_1st_stg.pickle")
logistic_model_2 = joblib.load("/kaggle/working/logistic_regression_as_1st_stg.pickle")
count_vector_step_1 = joblib.load("/kaggle/working/logistic_regression_cc_2nd_stg.pickle")
# count_vector_step_2 = joblib.load("output/count_vector_step_2.pkl")
# fewer_class_dict = joblib.load("output/fewer_class_dictionary.pkl")

# Prediction Function
def predict_cc(model_1, model_2,  query):
    # Predict using step 1 model
    test_1 = logistic_model_1.transform([query])
    y_pred = model_1.predict(test_1)
    if y_pred[0] == 'med':  # If classified as 'med', use step 2 model
        test_2 = count_vector_step_1.transform([query])
        y_pred = model_2.predict(test_2)

    # Check for overrides in fewer class dictionary
    # if query in final_dict.keys():
    #     y_pred = final_dict[query]
    return y_pred[0]

# File Processing
if uploaded_file is not None:
    # Read the uploaded Excel file
    excel_data = pd.read_excel(uploaded_file)

    # Preprocess the data
    st.write("Preprocessing Data...")
    processed_data = pre_processing(excel_data)
    x_test = processed_data['Claim Description']

    # Make predictions
    st.write("Making Predictions...")
    final_result = [predict_cc(logistic_model_1, count_vector_step_1,  query) for query in x_test]

    # Add predictions to DataFrame
    processed_data['Predicted Coverage Code'] = final_result

    # Display the processed DataFrame
    st.dataframe(processed_data)

    # Provide download link for processed file
    link = get_binary_file_downloader_link(processed_data, 'processed_data.xlsx', 'Download Processed Data')
    st.markdown(link, unsafe_allow_html=True)

# Add completion message
if uploaded_file is not None:
    st.write("Processing Completed!")

# Prediction Function
def predict_as(model_1,  query):
    # Predict using step 1 model
    test_1 = count_vector_step_1.transform([query])
    y_pred = model_1.predict(test_1)

    return y_pred[0]

# File Processing
if uploaded_file is not None:
    # Read the uploaded Excel file
    excel_data = pd.read_excel(uploaded_file)

    # Preprocess the data
    st.write("Preprocessing Data...")
    processed_data = pre_processing(excel_data)
    x_test = processed_data['Claim Description']

    # Make predictions
    st.write("Making Predictions...")
    final_result = [predict_as(logistic_model_2,  query) for query in x_test]

    # Add predictions to DataFrame
    processed_data['Predicted Accidental Source'] = final_result

    # Display the processed DataFrame
    st.dataframe(processed_data)

    # Provide download link for processed file
    link = get_binary_file_downloader_link(processed_data, 'processed_data.xlsx', 'Download Processed Data')
    st.markdown(link, unsafe_allow_html=True)


2024-12-01 20:57:22.308 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:57:22.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:57:22.311 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:57:22.312 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:57:22.313 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:57:22.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:57:22.317 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
